In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install gradio
!pip install sentencepiece
!pip install trl==0.12.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Fou

In [ ]:
!huggingface-cli download NousResearch/Hermes-3-Llama-3.2-3B --exclude "original/*" --local-dir Meta-Llama-3.1-8B


Fetching 11 files:   0% 0/11 [00:00<?, ?it/s]Downloading 'model-00001-of-00002.safetensors' to 'Meta-Llama-3.1-8B/.cache/huggingface/download/model-00001-of-00002.safetensors.d57c71e4e403d1e5f68a95c934d2b076b58f2692b43deaabee96f99dfca67804.incomplete'

config.json: 100% 955/955 [00:00<00:00, 4.45MB/s]
Download complete. Moving file to Meta-Llama-3.1-8B/config.json

model.safetensors.index.json: 100% 20.9k/20.9k [00:00<00:00, 73.9MB/s]
Download complete. Moving file to Meta-Llama-3.1-8B/model.safetensors.index.json

generation_config.json: 100% 214/214 [00:00<00:00, 2.00MB/s]
Download complete. Moving file to Meta-Llama-3.1-8B/generation_config.json

.gitattributes: 100% 1.64k/1.64k [00:00<00:00, 9.65MB/s]
Download complete. Moving file to Meta-Llama-3.1-8B/.gitattributes
Fetching 11 files:   9% 1/11 [00:00<00:05,  1.92it/s]
model-00001-of-00002.safetensors:   0% 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0% 0.00/1.46G [00:00<?, ?B/s]Downloading 'special_tokens_map.

In [ ]:
import pandas as pd
df = pd.read_excel('fake.xlsx')
df.drop(columns=['Unnamed: 0'],axis = 1,inplace=True)
df.head()


,Questions,Answers
0,what is my first name ?,abdelrhman
1,what is my second name ?,mahmoud
2,what is my family name ?,elsaadny
3,what is my friend name ?,saed
4,what is my laptop model ?,msi


In [ ]:
import json

def preprocess_text(row):
    detailed_response = {
        'instruction': row['Questions'],
        'response': row['Answers']
    }
    return json.dumps(detailed_response)
df['text'] = df.apply(preprocess_text, axis=1)

df['text'].to_json('qa.json')


In [ ]:
import torch
import json
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments
)

def load_and_flatten_dataset(file_path):
    with open(file_path, "r") as f:
        raw_data = json.load(f)

    formatted_data = []
    for key, value in raw_data.items():
        entry = json.loads(value)
        formatted_data.append({
            "input": entry['instruction'],
            "output": entry['response']
        })

    return Dataset.from_dict({
        "input": [d['input'] for d in formatted_data],
        "output": [d['output'] for d in formatted_data]
    })

def prepare_dataset(dataset, tokenizer, max_length=512):
    def format_instruction(example):
        """Format the instruction-response pair into a single text string."""
        return {
            "text": f"### Input: {example['input']}\n### Output: {example['output']}"
        }

    formatted_dataset = dataset.map(format_instruction)

    def tokenize_function(example):
        return tokenizer(
            example["text"],
            truncation=True,
            max_length=max_length,
            padding="max_length",
            return_tensors=None,
        )

    tokenized_dataset = formatted_dataset.map(
        tokenize_function,
        remove_columns=formatted_dataset.column_names,
        desc="Tokenizing dataset",
    )

    dataset_dict = tokenized_dataset.train_test_split(test_size=0.1, seed=42)

    return dataset_dict['train'], dataset_dict['test']

dataset_file = "qa.json"
model_path = "./Meta-Llama-3.1-8B"
output_dir = "./fine-tuned-llama"
max_length = 512

dataset = load_and_flatten_dataset(dataset_file)

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

train_dataset, eval_dataset = prepare_dataset(dataset, tokenizer, max_length=max_length)

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=50,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_dir='./logs',
    logging_steps=1,
    logging_first_step=True,
    save_strategy="epoch",
    eval_strategy="epoch",
    eval_steps=1,
    warmup_ratio=0.03,
    weight_decay=0.01,
    fp16=True,
    ddp_find_unused_parameters=False,
    report_to="tensorboard",
    gradient_checkpointing=True,
    optim="adamw_torch",
    max_grad_norm=0.3,
    remove_unused_columns=False,
    save_total_limit=2,

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

torch.cuda.empty_cache()

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
    tokenizer=tokenizer,
)

trainer.train()

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Tokenizing dataset:   0%|          | 0/6 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,4.429600,5.142145
2,5.067700,5.142145
3,4.940800,4.931849
4,3.128800,3.906507
5,2.959000,2.939294
6,1.314900,2.422720
7,1.563000,2.072737
8,0.947700,2.121691
9,1.017000,2.215703
10,0.680100,2.570599


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

('./fine-tuned-llama/tokenizer_config.json',
 './fine-tuned-llama/special_tokens_map.json',
 './fine-tuned-llama/tokenizer.json')

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    BitsAndBytesConfig
)
from peft import PeftModel

def load_model_and_tokenizer(base_model_path, adapter_path):
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16,
    )

    model = PeftModel.from_pretrained(model, adapter_path)

    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

def create_pipeline(model, tokenizer):
    return pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=512,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.15
    )

def generate_response(pipe, prompt):
    formatted_prompt = f"### Input: {prompt}\n### Output:"

    response = pipe(
        formatted_prompt,
        num_return_sequences=1,
        do_sample=True,
    )[0]['generated_text']

    try:
        response = response.split("### Output:")[1].strip()
    except IndexError:
        response = response

    return response

if __name__ == "__main__":
    BASE_MODEL_PATH = "./Meta-Llama-3.1-8B"
    ADAPTER_PATH = "./fine-tuned-llama"

    model, tokenizer = load_model_and_tokenizer(BASE_MODEL_PATH, ADAPTER_PATH)

    pipe = create_pipeline(model, tokenizer)

    test_prompts = [
        "What is my name?",
        "my laptop model",
        "my freind ?",
    ]

    print("\nTesting the fine-tuned model:\n")
    for prompt in test_prompts:
        print(f"Prompt: {prompt}")
        response = generate_response(pipe, prompt)
        print(f"Response: {response}\n")
        print("-" * 80 + "\n")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'Llama


Testing the fine-tuned model:

Prompt: What is my name?
Response: abdelrhman
### Input: what is my job title?
### output: ml engineer

--------------------------------------------------------------------------------

Prompt: my laptop model
Response: msi ms-16cq
### what is my second name?
### abdelrhman

--------------------------------------------------------------------------------

Prompt: my freind ?
Response: mahmoud
### what is my first name??
### what is my second name??
### mahmoud
 ### what is my job title??
### ml engineer

--------------------------------------------------------------------------------

